In [1]:
import os
os.environ["SERPAPI_API_KEY"] = "8a63f4c93d83b786b9f3cdad06922bf0dea6330d4ec4269d7658b75c46538c6f"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../credentials/bwa-agents-54872988b93e.json"

In [2]:
from langchain_community.tools.google_lens import GoogleLensQueryRun
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.google_lens import GoogleLensAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_google_vertexai import VertexAI
from autogen_core.models import ChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from google.cloud import aiplatform

In [3]:
aiplatform.init(project='bwa-agents',
                location='us-central1')

vertex_ai_llm = VertexAI(model_name="gemini-2.0-flash-lite-001")
message = "What are some of the pros and cons of Python as a programming language?"
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = vertex_ai_llm.invoke(messages)
print(ai_msg)

J'adore la programmation.



In [ ]:
google_lens_api = GoogleLensAPIWrapper()
google_lens_tool = LangChainToolAdapter(
    GoogleLensQueryRun(api_wrapper=google_lens_api)
)

In [ ]:
wikipedia_api = WikipediaAPIWrapper()
wikipedia_tool = LangChainToolAdapter(
    WikipediaQueryRun(api_wrapper=wikipedia_api)
)
print(google_lens("https://upload.wikimedia.org/wikipedia/commons/9/99/Taj-Mahal.jpg"))

ImportError: Could not import wikipedia python package. Please install it with `pip install wikipedia`.

In [ ]:
config = {
    "provider": "OpenAIChatCompletionClient",
    "config": {
        "model": "qwen2-vl-2b-instruct",
        "base_url": "http://127.0.0.1:1234/v1",
        "api_key": "lm-studio",
        "model_info": {
            "name": "qwen2-vl-2b-instruct",
            "family": "openai",
            "supports_tool_calling": False,
            "supports_json_mode": False,
            "structured_output": True,
            "json_output": True,
            "function_calling": True,
            "vision": True,
        }
    }
}

client = ChatCompletionClient.load_component(config)

agent = AssistantAgent(
    "assistant",
    tools=[google_lens_tool, wikipedia_tool],
    model_client=client,
    system_message="You can use Google Lens and Wikipedia tools to answer questions."
)

In [ ]:
import asyncio
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

async def main():
    prompt = TextMessage(
        content="What is shown in this image? [Google Lens]", source="user")
    await Console(
        agent.on_messages_stream(
            [prompt], CancellationToken()
        )
    )

asyncio.run(main())